In [1]:
import pathlib
import collections
import math
import numpy as np
import itertools

In [2]:
testlines = pathlib.Path('day19sample.txt').read_text().splitlines()

In [3]:
puzzlelines = pathlib.Path('day19.txt').read_text().splitlines()

## part 1 ##

In [4]:
def parse(lines):
    scanners = collections.defaultdict(list)
    for line in lines:
        if '--' in line:
            _, _, num, _ = line.split()
            curr_scanner = int(num)
        elif line:
            x, y, z = line.split(',')
            scanners[curr_scanner].append((int(x), int(y), int(z)))
        else:
            continue
    return scanners

In [5]:
def distance(pt1, pt2):
    return sum(abs(pt1[i]-pt2[i]) for i in range(3))

In [6]:
def get_all_pair_distances(pts):
    numpts = len(pts)
    d = collections.defaultdict(dict)
    for i in range(numpts-1):
        ipt = pts[i]
        for j in range(i+1, numpts):
            jpt = pts[j]
            distij = distance(ipt, jpt)
            d[ipt][jpt] = distij
            d[jpt][ipt] = distij
    return d        

In [7]:
def get_nearest_neighbor_distances(pairdict):
    nndists = set()
    for pt in pairdict:
        nndists.add(min(pairdict[pt].values()))
    return nndists

In [8]:
def neighbors(scanners):
    nlist = {}
    for scanner in scanners:
        pairdict = get_all_pair_distances(scanners[scanner])
        nlist[scanner] = get_nearest_neighbor_distances(pairdict)
    return nlist

In [9]:
scanners = parse(testlines)

In [10]:
nlist = neighbors(scanners)

In [11]:
num_scanners = len(nlist)
for i in range(1,num_scanners):
    print(i, len(set.intersection(nlist[0], nlist[i])))

1 8
2 2
3 0
4 4


In [12]:
for i in range(2,num_scanners):
    print(i, len(set.intersection(nlist[1], nlist[i])))

2 5
3 8
4 8


In [13]:
shared = set.intersection(nlist[0], nlist[1])
shared

{75, 95, 97, 141, 183, 190, 209, 245}

In [14]:
sd = []
for scanner,pts in scanners.items():
    sd.append(get_all_pair_distances(pts))

In [15]:
def find_basis_points(distances, shared):
    found = False
    for firstpt, d in distances.items():
        basis = [firstpt]
        for pt, dist in d.items():
            if dist in shared:
                basis.append(pt)
        if len(basis) >= 3:
            found = True
            break
    if not found:
        return None
    for firstpt, d in distances.items():
        for pt, dist in d.items():
            if pt in basis:
                continue
            if dist in shared:
                basis.append(pt)
                return basis
    return None

In [16]:
basis0 = find_basis_points(sd[0], shared)
basis0

[(404, -588, -901), (390, -675, -793), (544, -627, -890), (459, -707, 401)]

In [17]:
def match_points(basisa, sda, sdb):
    origina = basisa[0]
    joined_dists = [sda[origina][pt] for pt in basisa[1:3]]
    found = False
    for firstpt, d in sdb.items():
        if set(joined_dists) <= set(d.values()):
            # found the origin point
            found = True
            basisb = [firstpt]
            for secondpt, dist in d.items():
                if dist == joined_dists[0]:
                    basisb.append(secondpt)
                    break
            for secondpt, dist in d.items():
                if dist == joined_dists[1]:
                    basisb.append(secondpt)
            break
    if not found:
        return None
    lastptadists = sda[basisa[-1]]
    last_point_dists = set(lastptadists[pt] for pt in basisa[:-1])
    found = False
    for firstpt, d in sdb.items():
        if last_point_dists <= set(d.values()):
            # found the last point
            found = True
            basisb.append(firstpt)
            break
    if not found:
        return None
    return basisb

In [18]:
origina = basis0[0]
sd[0][origina][basis0[2]]

190

In [19]:
basis1 = match_points(basis0, sd[0], sd[1])
basis1

[(-336, 658, 858), (-322, 571, 750), (-476, 619, 847), (-391, 539, -444)]

In [20]:
def get_transition_matrix(basis):
    origin = np.array(basis[0])
    vecs = [np.array(pt) - origin for pt in basis[1:]]
    mat = np.zeros((3,3))
    for i, vec in enumerate(vecs):
        mat[:,i] = vec
    return mat

In [21]:
mat0 = get_transition_matrix(basis0)
mat1 = get_transition_matrix(basis1)

In [22]:
mat1inv = np.linalg.inv(mat1)
m0m1inv = np.matmul(mat0, mat1inv)
m0m1inv

array([[-1.00000000e+00,  3.90312782e-18,  2.92734587e-18],
       [-2.38795528e-17,  1.00000000e+00,  2.39933941e-16],
       [ 1.02457105e-17, -9.54097912e-18, -1.00000000e+00]])

In [23]:
mat0

array([[ -14.,  140.,   55.],
       [ -87.,  -39., -119.],
       [ 108.,   11., 1302.]])

In [24]:
mat1

array([[   14.,  -140.,   -55.],
       [  -87.,   -39.,  -119.],
       [ -108.,   -11., -1302.]])